In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib qt5

In [3]:
# DEFINE CONSTANTS
k = 1.381E-23 #J/K 
h = 6.626E-34 #J s 
B0 = 5.266

# PARTITION FUNCTION (WE WILL INTERPOLATE WITHIN THESE VALUES)
Qcdms = [13.8355, 28.4924, 64.0955, 164.3168, 449.0811, 1267.6705, 2628.0493, 3811.9477, 5277.5726]
T = [2.725, 5.0, 9.375, 18.75, 37.5, 75.0, 120.0, 150.0, 180.0]

Qfit = np.polyfit(T, Qcdms, 2)
Q = np.poly1d(Qfit)

T_array = np.linspace(0,200,1000)


plt.scatter(T, Qcdms, color="blue", s=50)
plt.plot(T_array, Q(T_array), color="blue", linestyle="--")

plt.ylabel("$Q_{rot}$", fontsize=15)
plt.xlabel("$T (K)$", fontsize=15)

plt.show()

In [4]:
molecule = pd.read_csv("CH3CN_fit_1.csv", delimiter="\t")

In [5]:
molecule

,Eup (K),Ln(Nu/gu),err(Ln(Nu/gu))
0,20.389921,20.580379,0.147003
1,13.244174,20.574362,0.134370
2,25.687393,19.776127,0.173957
3,18.541808,19.866676,0.229724
4,15.975260,20.814805,0.187116
5,8.829497,21.525804,0.191800


In [6]:
plt.scatter(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], c="black")
plt.errorbar(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], yerr=molecule["err(Ln(Nu/gu))"], fmt="none", ecolor="black", 
             elinewidth=1, capsize=3)
plt.ylabel("$\ln (N_u / g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()

In [7]:
from numpy import exp, linspace, pi, random, sign, sin

from lmfit import Parameters, fit_report, minimize


def residual(pars, x, data=None, eps=None):

    vals = pars.valuesdict()
    logN = vals['logN']
    Tex = vals['Tex']

    model = logN - x/Tex
  
    if data is None:
        return model
    if eps is None:
        return model - data
    return (model-data) / eps

x = molecule["Eup (K)"]

data = molecule["Ln(Nu/gu)"]
eps = molecule["err(Ln(Nu/gu))"]

fit_params = Parameters()
fit_params.add('logN', value=data[0])
fit_params.add('Tex', value=1e-1, min=0)

out = minimize(residual, fit_params, args=(x,), kws={'data': data, 'eps': eps})

print(fit_report(out))
# <end examples/doc_fitting_withreport.py>

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 34
    # data points      = 6
    # variables        = 2
    chi-square         = 18.3310237
    reduced chi-square = 4.58275592
    Akaike info crit   = 10.7010126
    Bayesian info crit = 10.2845316
[[Variables]]
    logN:  21.9406002 +/- 0.50617392 (2.31%) (init = 20.58038)
    Tex:   12.2019587 +/- 4.22308032 (34.61%) (init = 0.1)
[[Correlations]] (unreported correlations are < 0.100)
    C(logN, Tex) = -0.956


In [8]:
out.params.pretty_print()

Name     Value      Min      Max   Stderr     Vary     Expr Brute_Step
Tex       12.2        0      inf    4.223     True     None     None
logN     21.94     -inf      inf   0.5062     True     None     None


In [12]:
Eu = np.array(np.linspace(5,30,1000))

logN = out.params["logN"]
Tex = out.params["Tex"]

logN_err = 0.5062 
Tex_err = 4.223

bestfit = logN - Eu/Tex

# UNCERTAINTY ESTIMATE (# 1 SIGMA)
dy = np.sqrt(logN_err**2 + (Eu*Tex_err*Tex**(-2))**2)

plt.fill_between(Eu, bestfit+dy, bestfit-dy, color="blue", alpha=0.1)

plt.plot(Eu, bestfit, linestyle="--", c="red", linewidth=2)

plt.scatter(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], c="black")
plt.errorbar(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], yerr=molecule["err(Ln(Nu/gu))"], fmt="none", ecolor="black", 
             elinewidth=1, capsize=3)
plt.ylabel("$\ln (N_u / g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.title("$CH_3CN$")
plt.show()

## Calculate Column Density

In [13]:
Qrot = Q(Tex)
Qrot_err = np.sqrt( (Q[1]*Tex_err)**2 + (2*Q[2]*Tex_err*Tex)**2 )

logNtot = logN + np.log(Qrot)
logNtot_err = np.sqrt((logN_err)**2 + (Qrot_err/Qrot)**2)

In [14]:
plt.subplot2grid((1,5),(0,0), colspan=4)

plt.fill_between(Eu, bestfit+dy, bestfit-dy, color="blue", alpha=0.1)

plt.plot(Eu, bestfit, linestyle="--", c="red", linewidth=2)

plt.scatter(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], c="black")
plt.errorbar(molecule["Eup (K)"], molecule["Ln(Nu/gu)"], yerr=molecule["err(Ln(Nu/gu))"], fmt="none", ecolor="black", 
             elinewidth=1, capsize=3)
plt.ylabel("$\ln (N_u / g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.title("$CH_3CN$")

ax = plt.subplot2grid((1,5),(0,4), colspan=1)

plt.errorbar(0,logNtot, yerr=logNtot_err, ecolor="black", elinewidth=1, capsize=3)
plt.scatter(0,logNtot, color="black")
plt.xlim(-10,10)
plt.xticks([])
plt.ylim(logNtot-5*logNtot_err,logNtot+2*logNtot_err)
ax.yaxis.tick_right()
ax.set_ylabel("$\ln(N_{tot})$", fontsize=15)
ax.yaxis.set_label_position("right")

plt.show()

### Rotational Diagram Formatting (for CASSIS) -- ignore

In [ ]:
rotdiag = pd.read_csv("CH3CN_ROT.rotd.txt", sep="\t", header=1, skiprows=[2])

In [ ]:
rotdiag[rotdiag["IntensityMax"]>0.025]

In [ ]:
rotdiag = pd.read_csv("CH3CN_ROT.rotd.txt", sep="\t", header=1)